In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.layers import Dense, LeakyReLU, BatchNormalization, Flatten, Reshape, Input
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.datasets import mnist

In [0]:
class GAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        optimizer = Adam(0.0002, 0.5)

        self.discriminator = self.build_discriminator()

        self.discriminator.compile(
            loss = 'binary_crossentropy',
            optimizer = optimizer,
            metrics = ['accuracy']
        )

        self.generator = self.build_generator()
        self.generator.compile(
            loss = 'binary_crossentropy', 
            optimizer = optimizer
        )

        z = Input(shape=(100,))
        img = self.generator(z)

        self.discriminator.trainable = False

        valid = self.discriminator(img)

        self.combined = Model(z, valid)
        self.combined.compile(
            loss = 'binary_crossentropy', 
            optimizer = optimizer
        )

    def build_generator(self):

        noise_shape = (100,)

        model = Sequential()

        model.add(Dense(128, input_shape = noise_shape))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(BatchNormalization(momentum = 0.8))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(BatchNormalization(momentum = 0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(BatchNormalization(momentum = 0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=noise_shape)
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        img_shape = (self.img_rows, self.img_cols, self.channels)

        model = Sequential()

        model.add(Flatten(input_shape=img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape = img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50):

        (X_train, _), (_, _) = mnist.load_data()

        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)

        half_batch = int(batch_size / 2)

        for epoch in range(epochs):
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (half_batch, 100))

            gen_imgs = self.generator.predict(noise)

            d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1))) # 1-loss kinda thing
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            noise = np.random.normal(0, 1, (batch_size, 100))

            valid_y = np.array([1] * batch_size)

            g_loss = self.combined.train_on_batch(noise, valid_y)

            print ("%d [Dis loss: %f, acc.: %.2f%%] [Gen loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, 100))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("/content/drive/My Drive/own/GANs/images/mnist_%d.png" % epoch)
        plt.close()

In [18]:
gan = GAN()
gan.train(epochs = 30000, batch_size = 32, save_interval = 200) 

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
________________________________________________

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'



0 [D loss: 0.621812, acc.: 75.00%] [G loss: 0.718321]


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Streaming output truncated to the last 5000 lines.
25001 [D loss: 0.725805, acc.: 46.88%] [G loss: 0.839848]
25002 [D loss: 0.678413, acc.: 59.38%] [G loss: 0.939674]
25003 [D loss: 0.593732, acc.: 68.75%] [G loss: 0.869851]
25004 [D loss: 0.681235, acc.: 50.00%] [G loss: 0.863466]
25005 [D loss: 0.695317, acc.: 50.00%] [G loss: 0.848831]
25006 [D loss: 0.671855, acc.: 59.38%] [G loss: 0.804798]
25007 [D loss: 0.646056, acc.: 62.50%] [G loss: 0.845342]
25008 [D loss: 0.760344, acc.: 46.88%] [G loss: 0.835645]
25009 [D loss: 0.698145, acc.: 62.50%] [G loss: 0.818680]
25010 [D loss: 0.751165, acc.: 37.50%] [G loss: 0.763672]
25011 [D loss: 0.745896, acc.: 43.75%] [G loss: 0.909403]
25012 [D loss: 0.627299, acc.: 71.88%] [G loss: 0.787602]
25013 [D loss: 0.597189, acc.: 65.62%] [G loss: 0.797506]
25014 [D loss: 0.845032, acc.: 31.25%] [G loss: 0.782349]
25015 [D loss: 0.752583, acc.: 46.88%] [G loss: 0.829036]
25016 [D loss: 0.713399, acc.: 50.00%] [G loss: 0.832777]
25017 [D loss: 0.6704